# 4.06 Combining and Exporting Data (Part 02)

## 03 Importing libraries and .pkl merged file as df
## 04 Check the shape pf the dataframe
## 05 Combine merged dataframe with the products dataframe
## 06 Confirm redsults using _\_merge_ flag
## 07 Export new df as .pkl

## 03 Importing libraries and .pkl merged file

In [1]:
import numpy as np
import pandas as pd
import os

pathData = r'C:\Users\Michael\Desktop\Career Foundry\02 Data Immersion Course\04 Python Fundamentals for Data Analysts\Instacart Basket Analysis 2023 11\02 Data'

df_orders_products_combined = pd.read_pickle(os.path.join(pathData, 'Prepared Data', 'orders_products_combined.pkl'))

## 04 Check the shape pf the dataframe

In [3]:
df_orders_products_combined.shape

(32434489, 10)

Shape is correct

## 05 Combine merged dataframe with the products dataframe

### Import the products_checked.csv

In [4]:
df_prods = pd.read_csv(os.path.join(pathData, 'Prepared Data', 'products_checked.csv'), index_col = False)

### check the columns of the two dfs to be merged to find suitable merge on column

In [5]:
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [6]:
df_orders_products_combined.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


Shared column is 'product_id'
Left merge the df_orders_products_combined and df_prods
Left merge in order to check if there are any orders with product_ids not in the catalogue
Perform a check on the merge type after the merge

In [7]:
## delete the '_merge' column, this is no longer required
## a new _merge will be made for the next combined df
df_orders_products_combined = df_orders_products_combined.drop(columns = ['_merge'])

In [8]:
## check the type of the product_id columns
print(df_prods['product_id'].dtype)
print(df_orders_products_combined['product_id'].dtype)

int64
object


In [9]:
# ensure that product_id is str for both before merging
df_prods['product_id'] = df_prods['product_id'].astype('str')

### Merge Dataframes, initial merge is an outer merge
### in order to see if there are any mismatched rows, and why

In [10]:
# merge using outer join so that missing infomration can be reviewed after
# subesquently it might be replaced by a left or inner join
df_orders_products_merged = df_orders_products_combined.merge(df_prods, on = ['product_id'], how = 'outer', indicator = True)
df_orders_products_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1.0,2.0,8.0,NaN,196,1.0,0.0,Soda,77.0,7.0,9.0,both
1,2398795,1,2.0,3.0,7.0,15.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both
2,473747,1,3.0,3.0,12.0,21.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both
3,2254736,1,4.0,4.0,7.0,29.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both
4,431534,1,5.0,4.0,15.0,28.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both


In [13]:
# check the merge if there are any that come from only one of the dataframes
df_orders_products_merged['_merge'].value_counts()

# left means that the ordered product_id is not present in the list of products
# (i.e. a purchase made of an unknown product)

# Right means that a product is in the inventory catalogue, but has not been puchased

_merge
both          32404859
left_only        30200
right_only          11
Name: count, dtype: int64

In [19]:
# Get a count of all the rows that did not have a product_id asscociated with a product
df_orders_products_merged[df_orders_products_merged['_merge'] == 'left_only']['product_id'].value_counts(dropna = True)

product_id
1511     13397
34        6536
116       4359
6799      1978
4790      1804
2240      1689
262        179
3230        55
26519       51
1780        39
2586        29
69          19
525         18
4283        17
40440       13
3736         8
3159         5
38183        4
Name: count, dtype: int64

### Satisfied that missing row data not problematic to future analyese
### change the merged df to an inner joined frame, omitting the irrelevant rows

In [26]:
# Redo the merge, but on an inner merge to remove the unwanted data
df_orders_products_merged = df_orders_products_combined.merge(df_prods, on = ['product_id'], how = 'inner', indicator = True)
df_orders_products_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


## 06 check the _/_merge_ flag on the inner joined df

In [27]:
# redo the _merge flag count
df_orders_products_merged['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [28]:
# check the shape of the new merged dataframe
df_orders_products_merged.shape

(32404859, 14)

The new dataframe contains all the expected information (exludes those without a product name for the id)

## 07 Save the merged df as a .pkl

In [29]:
df_orders_products_merged.to_pickle(os.path.join(pathData,'Prepared Data', 'orders_products_merged.pkl'))